# Descriptive Statistics WSS

## Summary
This document explains how to merge the excel file with dataframe using pandas. After doing this, each poweroffovent will have a new feature named manufacturer. Additionally, new features such as skewness, kurtosis, and difference between the sensors are added.
In the result, a new dataset with features named itapudid, sensor1, sensor2, na_value_ratio1, na_value_ratio2, std, mean, max, skewness, kurtosis, differnce mean value, difference std, difference max, and  manufacturer will be presented. 

## Questions to be answered
- [Dead End] Does X predict Z?
- Does Y correlate with Z?

### Imports
Imports should be grouped in the following order:
1. Magics

2. Alphabetical order
    
    A. standard librarby imports
    
    B. related 3rd party imports
    
    C. local application/library specific imports

In [1]:
# Magics
%matplotlib inline
# Do below if you want interactive matplotlib plot ()
# %matplotlib notebook

# Reload modules before executing user code
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

# Show version information for dependency modules
# https://github.com/jrjohansson/version_information
%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas

Software versions
Python 3.5.2 64bit [MSC v.1900 64 bit (AMD64)]
IPython 5.1.0
OS Windows 7 6.1.7601 SP1
numpy 1.11.1
scipy 0.18.1
matplotlib 1.5.3
pandas 0.18.1
Fri Dec 09 12:30:11 2016 W. Europe Standard Time

In [1]:
# Standard library
import os
import sys
# sys.path.append('../src/')

# Third party imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math

# Local imports

In [2]:
# Customizations
sns.set() # matplotlib defaults

# Any tweaks that normally go in .matplotlibrc, etc., should be explicitly stated here
plt.rcParams['figure.figsize'] = (12,8)
%config InlineBackend.figure_format = 'retina'

### Load data

#### References
- df_s is the output of file: Data_Normolization_Speed_perSensor_01.ipynb
- df_m is the manufacturer information per itap

In [3]:
df = pd.read_pickle('../data/wss_n1')
df_s = pd.read_pickle('../data/wss_n_c1')
df_m = pd.read_pickle('../data/manufacturer')
df_na = pd.read_pickle('../data/na_ratio_wss_sensor1')

In [4]:
# This funtion is used to calculate the difference information on two sensors of each itap.
# The output columns include the mean value of difference, max values and standard deviation
def cal_dif(df):
    # Drop the records which has NA values, because you can't calculate numbers with NA value
    df = df.dropna(thresh = 4).reset_index(drop = True)
    df = df.fillna(0)
    df = pd.DataFrame(df, columns=['itapudid','max1stdetectwssc','max1stdetectwssd','max1stdetectwsse','max1stdetectwssf','difference'])

    # Calculate the difference of sensor detected values per poweroffevent
    for i in range(df.shape[0]):
        df.set_value(i,'difference',df.get_value(i,'max1stdetectwssc')-df.get_value(i,'max1stdetectwssd')+df.get_value(i,'max1stdetectwsse')-df.get_value(i,'max1stdetectwssf'))

    # Get the difference information per itapudid
    df = df[['itapudid','difference']].groupby('itapudid').describe().reset_index()
    df1 = df.itapudid
    df2 = df.difference
    dfc = pd.concat([df1, df2], axis=1)
    # The useful features can be mean value, max value and standard deviation
    dfc = dfc[['itapudid', 'mean', 'max', 'std']]

    dfc = dfc.rename(columns={'mean':'dif_mean', 'max':'dif_max', 'std':'dif_std'})
    
    return dfc

In [5]:
# This function is used to calculate the speed information per itap
# Which include mean value of pickup speed, max and standard deviation
# Also the skewness and kurtosis
def cal_speed(df_s):
    df_s_d = df_s.groupby('itapudid').describe().reset_index()
    df_s_1 = df_s_d.itapudid
    df_s_2 = df_s_d.speed
    df_s_d = pd.concat([df_s_1, df_s_2], axis=1)
    df_s_d = df_s_d[['itapudid', 'mean', 'max', 'std']]
    # Skewness and kurtosis
    df_k = df_s.groupby('itapudid').apply(pd.DataFrame.kurt).reset_index()
    df_k = df_k.rename(columns={'speed':'kurt'})
    df_sk = df_s.groupby('itapudid').skew().reset_index()
    df_sk = df_sk.rename(columns={'speed':'skew'})
    df_k_sk = pd.merge(df_k, df_sk, how='left', on='itapudid')
    
    df_s = pd.merge(df_s_d, df_k_sk, how='left', on='itapudid')
    
    return df_s

In [6]:
# Merge all the dataset with all features
def merge(df1, df2, df3, df4):
    df_mer = pd.merge(df1, df2, how='left', on='itapudid')
    df_mer = pd.merge(df_mer, df3, how='left', on='itapudid')
    df_mer = pd.merge(df_mer, df4, how='left', on='itapudid')
    
    return df_mer

In [7]:
df_final = merge(cal_speed(df_s), cal_dif(df), df_na, df_m)

In [8]:
df_final

,itapudid,mean,max,std,kurt,skew,dif_mean,dif_max,dif_std,NA_ratio,NA_ratio_dif,manufacturer
0,163540011001DC924C17D,5.161081,85.0,9.037809,18.046282,3.354441,-0.017297,3.0,0.651109,0.004306,0.000000,Knorr Bremse
1,163540014001DC924D43A,150.107713,250.0,102.053747,-1.379701,-0.627464,-0.196206,248.0,46.875689,0.018853,0.000531,NaN
2,163540015001DC924C174,127.994737,250.0,106.726353,-1.772258,-0.230970,0.804921,248.0,56.477145,0.258778,0.000000,Haldex
3,163540017001DC924DAFD,6.691897,92.0,10.586360,14.760282,3.122834,-0.048987,21.0,1.338518,0.007313,0.000000,Knorr Bremse
4,163540018001DC915C935,142.673547,250.0,101.989739,-1.501785,-0.521628,2.139130,206.0,54.287733,0.319157,0.000740,Haldex
5,163540020001DC924C70B,5.320965,86.0,10.398106,21.465616,3.899991,-0.037106,8.0,0.985746,0.002775,0.000000,Knorr Bremse
6,163540022001DC915C968,6.598689,88.0,11.177333,18.454150,3.570786,0.043478,37.0,1.406443,0.005491,0.000000,Knorr Bremse
7,163540023001DC924C17A,5.398860,78.0,9.236326,8.772556,2.623217,-0.009497,3.0,0.473544,0.005666,0.000000,Knorr Bremse
8,163540024001DC915C992,5.375199,83.0,9.896391,19.959377,3.685704,-0.059809,8.0,2.410160,0.004762,0.000000,Knorr Bremse
9,163540031001DC924C7B3,5.692063,87.0,11.105525,22.568146,4.140726,0.019048,2.0,0.406499,0.009434,0.000000,Knorr Bremse


In [19]:
df_final.to_pickle('../data/wss_all')